In [163]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

URL = 'https://data.census.gov/profile?q=United%20States&g=010XX00US'
driver = webdriver.Chrome()
driver.get(URL)

sleep(3)

Data = {"State": [], "Total_population": [], "Median_house_income": [],
        "Bachelor_degree_or_higher": [], "Employment Rate": [], "Total_Housing_Unit": [],
        "Uncoveraged_Health_Care": [], "Total_Employer_Establishements": [],
        "Total_HouseHold": [], "Hispanic_or_Latino(of_any_race)": []}

profile_list=[]

i = 0
while i < 6:
    driver.get(URL)
    sleep(3)

    Next_button = driver.find_element(By.XPATH, '//*[@id="content-layout"]/div[2]/div[1]/div/div[3]/main/div[1]/div[1]/div[2]/div[2]/div/div[2]/div[11]/div[1]/div[7]/div')
    sleep(3)   
    for _ in range(i):
        try:
            Next_button.click()
            sleep(1)
        except NoSuchElementException:
            print("No button to click. Ending the loop.")
            break
    i += 1
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'profile-name'))
    )  
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    States = soup.find_all("h2", class_="profile-name aqua-text-body")
    
    for State in States:
        state_text = State.text
        Data['State'].append(state_text)
    
    links = soup.find_all("a", class_="profile-link")
     
    for link in links:
        link_list = 'https://data.census.gov' + link.get('href')
        profile_list.append(link_list)



In [165]:
for each_url in profile_list:
    driver2 = webdriver.Chrome()
    driver2.get(each_url)
    sleep(12)
    new_html = driver2.page_source
    new_soup = BeautifulSoup(new_html, 'html.parser')
    sleep(2)
    try:
        Dataraw = new_soup.find_all("div", class_="highlight-value nowrap")[:9]
        
        Data['Total_population'].append(Dataraw[0].text.strip())
        Data['Median_house_income'].append(Dataraw[1].text.strip())
        Data['Bachelor_degree_or_higher'].append(Dataraw[2].text.strip())
        Data['Employment Rate'].append(Dataraw[3].text.strip())
        Data['Total_Housing_Unit'].append(Dataraw[4].text.strip())
        Data['Uncoveraged_Health_Care'].append(Dataraw[5].text.strip())
        Data['Total_Employer_Establishements'].append(Dataraw[6].text.strip())
        Data['Total_HouseHold'].append(Dataraw[7].text.strip())
        Data['Hispanic_or_Latino(of_any_race)'].append(Dataraw[8].text.strip())
    except IndexError:
        
        Data['Total_population'].append('N/A')
        Data['Median_house_income'].append('N/A')
        Data['Bachelor_degree_or_higher'].append('N/A')
        Data['Employment Rate'].append('N/A')
        Data['Total_Housing_Unit'].append('N/A')
        Data['Uncoveraged_Health_Care'].append('N/A')
        Data['Total_Employer_Establishements'].append('N/A')
        Data['Total_HouseHold'].append('N/A')
        Data['Hispanic_or_Latino(of_any_race)'].append('N/A')


In [133]:
import pandas as pd

In [168]:

df4 = pd.DataFrame.from_dict(Data)
csv_file_path = "E:/Resume/Census4.csv"
df4.to_csv(csv_file_path, index=False)
